# Finishing up this iteration of the dashboard

### Goal of this notebook 

All of this is great, but so far we have only made the controls for the dashboard. In this section we will finish up the rest.

We will mostly use the code from the earlier iteration of the dashboard, with the change that we will wrap the user elements inside of a class.

### Steps you will take in this notebook

1. Recreate the text widgets for the dashboard, this time as a class.
2. Make a class to hold the data table display and graph.
3. Explore one of the tricky bits of working with widget values that are mutable.

As a reminder, our final dashboard was a `Hbox` with the text and controls in a `VBox` on the left and another `VBox` with the data and plot. 

In [ ]:
#| default_exp widgets_classes

In [ ]:
#| export
import ipywidgets as widgets

## Implement the text widgets

The next cell defines the couple of bits of text that are needed.

In [ ]:
#| export

INTRO_TEXT = '''
<p><b>Curve Smoothing</b>
This tool is for smoothing and selecting land-ocean temperature data for visualization. Start by selecting a date
range, and then select the smoothing algorithm you want to use. Then click through to the next step, where you will change properies
of the curve smoothing algorithm you selected and visualize the data. 
</p>
'''

SOURCES_TEXT = '''
<p>
<b>About Land-Ocean Temperature Data</b>
<a href="https://climate.nasa.gov/vital-signs/global-temperature/"
target="_blank">Global Temperature (NASA)</a>
,
<a href="https://data.giss.nasa.gov/gistemp/"
target="_blank">GISS Surface Temperature Analysis (NASA)</a>
</p><p>
This site is based on data downloaded from the following site on 2020-07-14:
<a href="https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt"  # noqa
target="_blank">Global Mean Estimates based on Land and Ocean Data (NASA)</a>
'''


These are similar enough that we'll put them into a class of their own.

In [ ]:
#| export 

class TextBoxes(widgets.VBox):
    def __init__(self):
        # This calls the __init__ method of the parent class, which is 
        # widgets.VBox.
        super().__init__()
        # This layout is for the two HTML widgets
        layout = widgets.Layout(max_width='500px', margin='15px 0 15px 0')
        self.smoothing = widgets.HTML(value=INTRO_TEXT, layout=layout)
        self.sources = widgets.HTML(value=SOURCES_TEXT, layout=layout)

        # Layout for the VBox holding the HTML widgets
        self.layout = widgets.Layout(min_width='500px')
        self.children = [self.smoothing, self.sources]

Let's take a look to make sure this looks the same as it did before.

In [ ]:
TextBoxes()

## Implement plotting and data display

Next, we make the widget with the plot. To do that this time around we will make a class with a method to filter the data and to make the plot.

First, some imports.

In [ ]:
#| export
from pathlib import Path

from ipydatagrid import DataGrid
from matplotlib import pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
import traitlets as tr

Next, a small class that reads the data file and has a *traitlet* called `smoothing_info` that is a dictionary. Recall that the `traitlets` package is what implements the observability of Jupyter widgets. We use a dictionary here because the intent, in the next notebook, is to link the `smoothing_info` trait of the object below to the `_value` trait of the `ipyautotui`-generated widget.

In addition to declaring the traitlet we use `observe` in a new way, as a decorator for one of the methods of our class. This snippet of code means "call the method `select_and_plot` any time the value of `smoothing_info` changes":

```python

    @tr.observe('smoothing_info')
    def select_and_plot(self, change):
        self.window_size = ...
```

It is that method which does most of the work of generating a plot and updating the displayed data.

In [ ]:
#| export
DATA_DIR = 'data'
DATA_FILE = 'land-ocean-temp-index.csv'

class DataAndPlot(tr.HasTraits):
    smoothing_info = tr.Dict(allow_none=True, default=None)
    
    def __init__(self):
        self.original_data = pd.read_csv(Path(DATA_DIR) / DATA_FILE, escapechar='#')
        self.plot_output = widgets.Output()
        self.data_output = DataGrid(self.original_data, header_visibility="column", auto_fit_columns=True)

    @tr.observe('smoothing_info')
    def select_and_plot(self, change):
        self.window_size = change["new"]["window_size"]
        self.polynomial_order = change["new"]["polynomial_order"]
        self.year_range = change["new"]["year_range"]
        self.original_data['Savitzky-Golay'] = savgol_filter(
            self.original_data['Temperature'], 
            self.window_size, 
            self.polynomial_order
        )
        
        self.selected = (
            self.original_data[(self.original_data['Year'] >= self.year_range[0]) & 
                               (self.original_data['Year'] <= self.year_range[1])]
        )
        self.display_plot()
        self.display_data()

    def display_plot(self): 
        # This plotting function is copied from the first 
        # version of the dashboard.
        self.plot_output.clear_output(wait=True) 
        with self.plot_output: 
            plt.xlabel('Year') 
            plt.ylabel('Temperature Anomalies over Land w.r.t. 1951-80 (˚C)') 
            plt.title('Global Annual Mean Surface Air Temperature Change')
            plt.plot(self.selected['Year'], self.selected['Temperature'], label='Raw Data') 
            plt.plot(self.selected['Year'], self.selected['Savitzky-Golay'], label='Smoothed Data') 
            plt.legend()
            plt.show() 

    def display_data(self):
        self.data_output.data = self.selected

Let's make one of these objects and display its `plot_output`, which is an `Output` widget.

In [ ]:
dap = DataAndPlot()
dap.plot_output

Because our class observes the value of `smoothing_info`, setting `smoothing_info` should generate a plot.

In [ ]:
dap.smoothing_info = dict(year_range=(1950, 2000), window_size=5, polynomial_order=2)

## Mutable traits are a little tricky

One important thing to watch out for that applies to widgets broadly. When the value of a trait is a *mutable* type, i.e. a type whose contents can change, like a `list` or a `dict`, changing the values inside that mutable object will not trigger a change in the event.

To see that in action, in the cell below try changing `dap.smoothing_info["window_size"]` to a value large than 5, the value with used in the cell above. Pick a fairly large value so that any change in the graph will be obvious

In [ ]:
# TODO: write answer 

dap.smoothing_info["window_size"] = 8

To trigger a change you must set `smoothing_info` to a new dictionary. This also applies to widgets generated with `ipyautoui` since their values are always dictionaries.

One shortcut for dong this is in the cell below. We get a copy of the current `smoothing_info`, change the value we want to change in the copy, then set `smoothing_info` equal to the copy.

In [ ]:
current_smoothing = dap.smoothing_info.copy()
current_smoothing["window_size"] = 20
dap.smoothing_info = current_smoothing

Finally, lets make sure the data display is what we had before.

In [ ]:
dap.data_output

Now that we are sure things are working well, we export this code with nbdev.

In [ ]:
from nbdev.export import nb_export

nb_export('03c_other_dashboard_elements.ipynb', 'dashboard_pydantic')